In [86]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from textblob import TextBlob
from wordcloud import WordCloud
from sklearn.inspection import permutation_importance
import itertools

In [87]:
file_path = 'wine_quality_1000.csv'  # Adjust file path if necessary
data = pd.read_csv(file_path, index_col = 0)

In [88]:

def prepare_tfidf_vectorizer(data, max_features=100):
    tfidf_vectorizer = TfidfVectorizer(max_features=max_features)
    tfidf_vectorizer.fit(data['description'].fillna(""))  # Fit on full dataset or train_val only
    return tfidf_vectorizer

In [89]:
def prepare_onehot_encoder(data, column='variety'):
    encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    encoder.fit(data[[column]].fillna(""))  # Fit on the specified column
    return encoder

In [90]:
# Updated preprocess_data function
def preprocess_data(
    data,
    tfidf_vectorizer,
    onehot_encoder,
    text_vectorization=True,
    text_length=True,
    sentiment_analysis=True,
    price_per_point=True,
    log_price=True,
    point_binning=True,
    one_hot_encode=True,
    frequency_encode=True,
    price_variety_interaction=True,
    clustering=True,
):
    # Text Vectorization
    if text_vectorization and tfidf_vectorizer:
        tfidf_matrix = tfidf_vectorizer.transform(data['description'].fillna(""))  # Transform with pre-fitted vectorizer
        tfidf_df = pd.DataFrame(
            tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out(), index=data.index
        )
        data = pd.concat([data, tfidf_df], axis=1)  # Concatenate along columns

    # Text Length
    if text_length:
        data['description_length'] = data['description'].apply(lambda x: len(str(x)))

    # Sentiment Analysis
    if sentiment_analysis:
        data['sentiment_polarity'] = data['description'].apply(
            lambda x: TextBlob(str(x)).sentiment.polarity
        )

    # Price per Point
    if price_per_point:
        data['price_per_point'] = data['price'] / data['points']

    # Log Transformation of Price
    if log_price:
        data['log_price'] = np.log1p(data['price'])

    # Point Binning
    if point_binning:
        data['point_category'] = pd.cut(
            data['points'], bins=[0, 85, 90, 95, 100], labels=['Low', 'Medium', 'High', 'Exceptional']
        )

    # One-Hot Encoding with Pre-Fitted Encoder
    if one_hot_encode and onehot_encoder:
        encoded_columns = onehot_encoder.transform(data[['variety']].fillna(""))
        encoded_df = pd.DataFrame(
            encoded_columns, columns=onehot_encoder.get_feature_names_out(), index=data.index
        )
        data = pd.concat([data, encoded_df], axis=1)

    # Frequency Encoding
    if frequency_encode:
        variety_counts = data['variety'].value_counts()
        data['variety_frequency'] = data['variety'].map(variety_counts)

    # Price-Variety Interaction
    if price_variety_interaction:
        variety_avg_price = data.groupby('variety')['price'].transform('mean')
        data['price_relative_to_variety'] = data['price'] / variety_avg_price

    # Clustering
    if clustering:
        kmeans = KMeans(n_clusters=5, random_state=42)
        data['wine_cluster'] = kmeans.fit_predict(data[['price', 'points']].fillna(0))

    return data

In [91]:
tfidf_vectorizer = prepare_tfidf_vectorizer(data)
onehot_encoder = prepare_onehot_encoder(data, column='variety')


In [93]:
processed_data = preprocess_data(
    data,
    tfidf_vectorizer,
    onehot_encoder,
    text_vectorization=True,
    text_length=True,
    sentiment_analysis=True,
    price_per_point=True,
    log_price=True,
    point_binning=True,
    one_hot_encode=True,
    frequency_encode=False,
    price_variety_interaction=True,
    clustering=True,
)

In [94]:
processed_data.head()

,country,description,points,price,variety,acidity,all,an,and,apple,...,variety_Turbiana,variety_Verdejo,variety_Verdelho,variety_Viognier,variety_Viognier-Valdiguié,variety_White Blend,variety_White Riesling,variety_Zinfandel,price_relative_to_variety,wine_cluster
87131,US,"Very good Dry Creek Zin, robust and dry and sp...",89,25.0,Zinfandel,0.000000,0.0,0.0,0.232413,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.964630,0
57952,France,A very herbaceous character makes this wine se...,84,20.0,Bordeaux-style White Blend,0.000000,0.0,0.0,0.290431,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.571429,0
96046,US,"A little simple and easy, but there's a wealth...",84,19.0,Rosé,0.244571,0.0,0.0,0.195024,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.045455,0
31920,US,From a dry-farmed vineyard and treated to wild...,88,38.0,Petite Sirah,0.000000,0.0,0.0,0.335920,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.384615,3
6091,US,"From a site near Annapolis, this wine shows a ...",91,62.0,Pinot Noir,0.000000,0.0,0.0,0.409043,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.387780,3


In [95]:
# Generate all hyperparameter combinations
preprocessing_hyperparams = {
    'text_vectorization': [True, False],
    'text_length': [True, False],
    'sentiment_analysis': [True, False],
    'price_per_point': [True, False],
    'log_price': [True, False],
    'point_binning': [True, False],
    'one_hot_encode': [True, False],
    'frequency_encode': [True, False],
    'price_variety_interaction': [True, False],
    'clustering': [True, False],
}


preprocessing_combinations = list(itertools.product(
    *[preprocessing_hyperparams[key] for key in preprocessing_hyperparams]
))

# Models to train
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
}

# Train-test split
processed_data = preprocess_data(data, tfidf_vectorizer, onehot_encoder, **{key: True for key in preprocessing_hyperparams})
features = processed_data.select_dtypes(include=[np.number])
target = processed_data['country']

X_train_val, X_test, y_train_val, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Hyperparameter testing
best_model = None
best_accuracy = 0
best_preprocessing_config = None
best_model_name = None

print("Evaluating all preprocessing feature combinations...\n")
for config in preprocessing_combinations:
    step_config = {key: value for key, value in zip(preprocessing_hyperparams.keys(), config)}

    # Preprocess train/validation data
    processed_train_val = preprocess_data(data.loc[X_train_val.index], tfidf_vectorizer, onehot_encoder, **step_config)
    X_train_val_processed = processed_train_val.select_dtypes(include=[np.number])
    y_train_val_processed = processed_train_val['country']

    # Split into train and validation
    X_train, X_val, y_train, y_val = train_test_split(X_train_val_processed, y_train_val_processed, test_size=0.2, random_state=42)

    for model_name, model in models.items():
        print(f"Training {model_name} with preprocessing configuration: {step_config}...")
        model.fit(X_train, y_train)
        y_val_pred = model.predict(X_val)
        accuracy = accuracy_score(y_val, y_val_pred)
        print(f"Validation Accuracy for {model_name}: {accuracy:.4f}")

        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_model = model
            best_preprocessing_config = step_config
            best_model_name = model_name
            print(f"New best model found: {model_name} with accuracy {best_accuracy:.4f}")


Evaluating all preprocessing feature combinations...

Training Logistic Regression with preprocessing configuration: {'text_vectorization': True, 'text_length': True, 'sentiment_analysis': True, 'price_per_point': True, 'log_price': True, 'point_binning': True, 'one_hot_encode': True, 'frequency_encode': True, 'price_variety_interaction': True, 'clustering': True}...


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Validation Accuracy for Logistic Regression: 0.8500
New best model found: Logistic Regression with accuracy 0.8500
Training Random Forest with preprocessing configuration: {'text_vectorization': True, 'text_length': True, 'sentiment_analysis': True, 'price_per_point': True, 'log_price': True, 'point_binning': True, 'one_hot_encode': True, 'frequency_encode': True, 'price_variety_interaction': True, 'clustering': True}...
Validation Accuracy for Random Forest: 0.7250
Training Logistic Regression with preprocessing configuration: {'text_vectorization': True, 'text_length': True, 'sentiment_analysis': True, 'price_per_point': True, 'log_price': True, 'point_binning': True, 'one_hot_encode': True, 'frequency_encode': True, 'price_variety_interaction': True, 'clustering': False}...


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Validation Accuracy for Logistic Regression: 0.8500
Training Random Forest with preprocessing configuration: {'text_vectorization': True, 'text_length': True, 'sentiment_analysis': True, 'price_per_point': True, 'log_price': True, 'point_binning': True, 'one_hot_encode': True, 'frequency_encode': True, 'price_variety_interaction': True, 'clustering': False}...
Validation Accuracy for Random Forest: 0.7438
Training Logistic Regression with preprocessing configuration: {'text_vectorization': True, 'text_length': True, 'sentiment_analysis': True, 'price_per_point': True, 'log_price': True, 'point_binning': True, 'one_hot_encode': True, 'frequency_encode': True, 'price_variety_interaction': False, 'clustering': True}...


...

Validation Accuracy for Logistic Regression: 0.6125
Training Random Forest with preprocessing configuration: {'text_vectorization': False, 'text_length': False, 'sentiment_analysis': False, 'price_per_point': False, 'log_price': False, 'point_binning': False, 'one_hot_encode': False, 'frequency_encode': True, 'price_variety_interaction': False, 'clustering': True}...
Validation Accuracy for Random Forest: 0.7125
Training Logistic Regression with preprocessing configuration: {'text_vectorization': False, 'text_length': False, 'sentiment_analysis': False, 'price_per_point': False, 'log_price': False, 'point_binning': False, 'one_hot_encode': False, 'frequency_encode': True, 'price_variety_interaction': False, 'clustering': False}...
Validation Accuracy for Logistic Regression: 0.6125
Training Random Forest with preprocessing configuration: {'text_vectorization': False, 'text_length': False, 'sentiment_analysis': False, 'price_per_point': False, 'log_price': False, 'point_binning': False,

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Validation Accuracy for Logistic Regression: 0.6188
Training Random Forest with preprocessing configuration: {'text_vectorization': False, 'text_length': False, 'sentiment_analysis': False, 'price_per_point': False, 'log_price': False, 'point_binning': False, 'one_hot_encode': False, 'frequency_encode': False, 'price_variety_interaction': True, 'clustering': True}...
Validation Accuracy for Random Forest: 0.6000
Training Logistic Regression with preprocessing configuration: {'text_vectorization': False, 'text_length': False, 'sentiment_analysis': False, 'price_per_point': False, 'log_price': False, 'point_binning': False, 'one_hot_encode': False, 'frequency_encode': False, 'price_variety_interaction': True, 'clustering': False}...


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Validation Accuracy for Logistic Regression: 0.6188
Training Random Forest with preprocessing configuration: {'text_vectorization': False, 'text_length': False, 'sentiment_analysis': False, 'price_per_point': False, 'log_price': False, 'point_binning': False, 'one_hot_encode': False, 'frequency_encode': False, 'price_variety_interaction': True, 'clustering': False}...
Validation Accuracy for Random Forest: 0.5563
Training Logistic Regression with preprocessing configuration: {'text_vectorization': False, 'text_length': False, 'sentiment_analysis': False, 'price_per_point': False, 'log_price': False, 'point_binning': False, 'one_hot_encode': False, 'frequency_encode': False, 'price_variety_interaction': False, 'clustering': True}...
Validation Accuracy for Logistic Regression: 0.6188
Training Random Forest with preprocessing configuration: {'text_vectorization': False, 'text_length': False, 'sentiment_analysis': False, 'price_per_point': False, 'log_price': False, 'point_binning': False

In [99]:
print("Best preprocessing configuration:", best_preprocessing_config)
processed_train_val = preprocess_data(data.loc[X_train_val.index], tfidf_vectorizer, onehot_encoder, **best_preprocessing_config)
X_final_train = processed_train_val.select_dtypes(include=[np.number])
y_final_train = processed_train_val['country']

processed_test = preprocess_data(data.loc[X_test.index], tfidf_vectorizer, onehot_encoder, **best_preprocessing_config)
X_test_processed = processed_test.select_dtypes(include=[np.number])
y_test_processed = processed_test['country']

#Final model training and evaluation
print("\nFinal training on train + validation set...")
best_model.fit(X_final_train, y_final_train)
y_test_pred = best_model.predict(X_test_processed)

print("\nFinal Test Metrics:")
print(classification_report(y_test_processed, y_test_pred))

Best preprocessing configuration: {'text_vectorization': True, 'text_length': True, 'sentiment_analysis': False, 'price_per_point': True, 'log_price': True, 'point_binning': True, 'one_hot_encode': True, 'frequency_encode': True, 'price_variety_interaction': False, 'clustering': True}

Final training on train + validation set...

Final Test Metrics:
              precision    recall  f1-score   support

      France       0.73      0.73      0.73        26
       Italy       0.59      0.78      0.67        37
       Spain       0.73      0.57      0.64        14
          US       0.88      0.81      0.84       123

    accuracy                           0.78       200
   macro avg       0.73      0.72      0.72       200
weighted avg       0.79      0.78      0.78       200



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [100]:
# Final results with all proprocessing steps activated
processed_data = preprocess_data(
    data,
    tfidf_vectorizer, onehot_encoder,
    text_vectorization=True,
    text_length=True,
    sentiment_analysis=True,
    price_per_point=True,
    log_price=True,
    point_binning=True,
    one_hot_encode=True,
    frequency_encode=True,
    price_variety_interaction=True,
    clustering=True,
)


features = processed_data.drop(['country'], axis=1)
features = processed_data.select_dtypes(include=[np.number])
target = processed_data['country']
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)


models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Random Forest': RandomForestClassifier(),
}

for name, model in models.items():
    print(f"\nTraining {name}...")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"Classification Report for {name}:\n")
    print(classification_report(y_test, y_pred))


Training Logistic Regression...


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Classification Report for Logistic Regression:

              precision    recall  f1-score   support

      France       0.83      0.38      0.53        26
       Italy       0.71      0.68      0.69        37
       Spain       1.00      0.50      0.67        14
          US       0.79      0.93      0.86       123

    accuracy                           0.79       200
   macro avg       0.83      0.62      0.69       200
weighted avg       0.79      0.79      0.77       200


Training Random Forest...
Classification Report for Random Forest:

              precision    recall  f1-score   support

      France       0.75      0.12      0.20        26
       Italy       0.76      0.51      0.61        37
       Spain       1.00      0.07      0.13        14
          US       0.71      0.98      0.82       123

    accuracy                           0.71       200
   macro avg       0.80      0.42      0.44       200
weighted avg       0.74      0.71      0.65       200

